# 我是说明，请阅读

我们需要的**原始数据**字段：**日期,广告形式,产品,展现量,点击量,点击率,总加购数,转化率,ROI,总订单行,总订单金额,总费用。**

我们建立的**因变量：点击率/转化率，销售额/ROI，点击量/展现量，加购数/点击量，订单数/加购数**

我们的**自变量：广告类型的0-1变量，每日每渠道的投入金额（总费用）**

**目前的数据频次为每天，如果要改数据频次，因变量不能直接加和，因为他们都是比率**

京东数据应该可以直接生成下来，天猫数据需要改代码里面一些变量的名称之后才能相除，或者也可以将天猫数据的column改成京东的样子（第一行原始数据字段）

**京东**

京东所有广告类型：PUSH海投,购物触点,海投计划，京东展位，京东直投，京选店铺，经典海投，快车，首焦海投

分成五类：
展位（京东展位）、触点（购物触点）、海投（PUSH海投，海投计划，经典海投，首焦海投）、快车（快车，京选店铺）、其他（京东直投）

**天猫**

天猫所有广告类型都是分成sheet放的，有四种：超级推荐，直通车，钻展，品销宝

或许需要手动加变量，格式：在每一页加上四列类型变量（是否为超级推荐，是否为直通车，是否为钻展，是否为品销宝；是的话用1，不是用0）

天猫每个sheet的变量不一样，需要分别处理：

超级推荐：时间，展现，点击，点击率，添加购物车量，点击转化率，投资回报率，成交订单量，成交订单金额，消耗

直通车：日期，展现量，点击量，点击率，总购物车数，点击转化率，投入产出比，总成交笔数，总成交金额(分)，花费(分)

钻展：日期，有效展现，有效点击数，点击率，添加购物车量，点击转化率，投资回报率，成交订单量，成交订单金额，消耗

品销宝：日期，展现量，点击量，点击率，宝贝加购数，转化率，回报率，成交笔数，成交金额，消耗

每个sheet都要加上这四个变量：是否为超级推荐，是否为直通车，是否为钻展，是否为品销宝，让该类别全部为1，其他全部为0


In [11]:
import pandas as pd
import numpy as np
df = pd.read_csv('京东每日数据求和-细分广告版.csv',encoding='gb18030')
df2 = df.loc[:,['日期','广告形式','产品','展现量','点击量','点击率','总加购数','转化率','ROI','总订单行','总订单金额','总费用']]
print(df2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35393 entries, 0 to 35392
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   日期      33049 non-null  object 
 1   广告形式    33049 non-null  object 
 2   产品      32409 non-null  object 
 3   展现量     33049 non-null  float64
 4   点击量     33049 non-null  float64
 5   点击率     33049 non-null  float64
 6   总加购数    33049 non-null  float64
 7   转化率     33049 non-null  float64
 8   ROI     33049 non-null  object 
 9   总订单行    33049 non-null  float64
 10  总订单金额   33049 non-null  float64
 11  总费用     32181 non-null  float64
dtypes: float64(8), object(4)
memory usage: 3.2+ MB
None


D:\Software\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (28,30,39,40,41,42,43,44,45,46) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [ ]:
#天猫数据适用
#df2.columns = ['日期','广告形式','产品','展现量','点击量','点击率','总加购数','转化率','ROI','总订单行','总订单金额','总费用']

In [12]:
df2 = df2.replace('#DIV/0!', np.NaN)
df2.loc[:,['ROI']] = pd.DataFrame(df2.loc[:,['ROI']],dtype = float)
print(df2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35393 entries, 0 to 35392
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   日期      33049 non-null  object 
 1   广告形式    33049 non-null  object 
 2   产品      32409 non-null  object 
 3   展现量     33049 non-null  float64
 4   点击量     33049 non-null  float64
 5   点击率     33049 non-null  float64
 6   总加购数    33049 non-null  float64
 7   转化率     33049 non-null  float64
 8   ROI     32268 non-null  float64
 9   总订单行    33049 non-null  float64
 10  总订单金额   33049 non-null  float64
 11  总费用     32181 non-null  float64
dtypes: float64(9), object(3)
memory usage: 3.2+ MB
None


效果衡量方式：

1.利用ROI

i.点击率/转化率

ii.销售额/ROI

2.利用各层转换率：点击量/展现量，加购数/点击量，订单数/加购数

i.PCA

ii.在三个中寻找最小的一个

In [13]:
df2

日期  广告形式     产品     展现量   点击量   点击率  总加购数  转化率  ROI  总订单行  \
0      2019/5/30  购物触点  SR501   893.0   4.0  0.45   0.0  0.0  0.0   0.0   
1      2019/5/30  购物触点  SR501  1327.0   8.0  0.60   1.0  0.0  0.0   0.0   
2      2019/5/29  购物触点  SR501   961.0  10.0  1.04   0.0  0.0  0.0   0.0   
3      2019/5/29  购物触点  SR501  1463.0  18.0  1.23   0.0  0.0  0.0   0.0   
4      2019/5/28  购物触点  SR501  1916.0  14.0  0.73   1.0  0.0  0.0   0.0   
...          ...   ...    ...     ...   ...   ...   ...  ...  ...   ...   
35388        NaN   NaN    NaN     NaN   NaN   NaN   NaN  NaN  NaN   NaN   
35389        NaN   NaN    NaN     NaN   NaN   NaN   NaN  NaN  NaN   NaN   
35390        NaN   NaN    NaN     NaN   NaN   NaN   NaN  NaN  NaN   NaN   
35391        NaN   NaN    NaN     NaN   NaN   NaN   NaN  NaN  NaN   NaN   
35392        NaN   NaN    NaN     NaN   NaN   NaN   NaN  NaN  NaN   NaN   

       总订单金额    总费用  
0        0.0   5.74  
1        0.0  15.51  
2        0.0  14.87  
3        0.0  34.20  
4        0.0  26.08  
...      ...    ...  
35388    NaN    NaN  
35389    NaN    NaN  
35390    NaN    NaN  
35391    NaN    NaN  
35392    NaN    NaN  

[35393 rows x 12 columns]

In [14]:
size=len(df2)

In [16]:
#京东所有广告类型：PUSH海投,购物触点,海投计划，京东展位，京东直投，京选店铺，经典海投，快车，首焦海投
#dummy1=展位（京东展位）
#dummy2=触点（购物触点）
#dummy3=海投（PUSH海投，海投计划，海投新品，经典海投，首焦海投）
#dummy4=快车（快车，京选店铺）
#dummy5=其他（京东直投，直投）
dummy1=[]
d1=['京东展位']
dummy2=[]
d2=['购物触点']
dummy3=[]
d3=['PUSH海投','海投计划','海投新品','经典海投','首焦海投']
dummy4=[]
d4=['快车','京选店铺']
dummy5=[]
d5=['京东直投','直投']
for i in range(size):
    if df2['广告形式'][i] in d1:
        dummy1.append(1)
        dummy2.append(0)
        dummy3.append(0)
        dummy4.append(0)
        dummy5.append(0)
    elif df2['广告形式'][i] in d2:
        dummy1.append(0)
        dummy2.append(1)
        dummy3.append(0)
        dummy4.append(0)
        dummy5.append(0)
    elif df2['广告形式'][i] in d3:
        dummy1.append(0)
        dummy2.append(0)
        dummy3.append(1)
        dummy4.append(0)
        dummy5.append(0)
    elif df2['广告形式'][i] in d4:
        dummy1.append(0)
        dummy2.append(0)
        dummy3.append(0)
        dummy4.append(1)
        dummy5.append(0)
    elif df2['广告形式'][i] in d5:
        dummy1.append(0)
        dummy2.append(0)
        dummy3.append(0)
        dummy4.append(0)
        dummy5.append(1)
dummy_1=pd.DataFrame(dummy1)
dummy_2=pd.DataFrame(dummy2)
dummy_3=pd.DataFrame(dummy3)
dummy_4=pd.DataFrame(dummy4)
dummy_5=pd.DataFrame(dummy5)
df2['是否为展位广告']=dummy_1
df2['是否为触点广告']=dummy_2
df2['是否为海投广告']=dummy_3
df2['是否为快车广告']=dummy_4
df2['是否为京东直投']=dummy_5
df2

日期  广告形式     产品     展现量   点击量   点击率  总加购数  转化率  ROI  总订单行  \
0      2019/5/30  购物触点  SR501   893.0   4.0  0.45   0.0  0.0  0.0   0.0   
1      2019/5/30  购物触点  SR501  1327.0   8.0  0.60   1.0  0.0  0.0   0.0   
2      2019/5/29  购物触点  SR501   961.0  10.0  1.04   0.0  0.0  0.0   0.0   
3      2019/5/29  购物触点  SR501  1463.0  18.0  1.23   0.0  0.0  0.0   0.0   
4      2019/5/28  购物触点  SR501  1916.0  14.0  0.73   1.0  0.0  0.0   0.0   
...          ...   ...    ...     ...   ...   ...   ...  ...  ...   ...   
35388        NaN   NaN    NaN     NaN   NaN   NaN   NaN  NaN  NaN   NaN   
35389        NaN   NaN    NaN     NaN   NaN   NaN   NaN  NaN  NaN   NaN   
35390        NaN   NaN    NaN     NaN   NaN   NaN   NaN  NaN  NaN   NaN   
35391        NaN   NaN    NaN     NaN   NaN   NaN   NaN  NaN  NaN   NaN   
35392        NaN   NaN    NaN     NaN   NaN   NaN   NaN  NaN  NaN   NaN   

       总订单金额    总费用  是否为展位广告  是否为触点广告  是否为海投广告  是否为快车广告  是否为京东直投  
0        0.0   5.74      0.0      1.0      0.0      0.0      0.0  
1        0.0  15.51      0.0      1.0      0.0      0.0      0.0  
2        0.0  14.87      0.0      1.0      0.0      0.0      0.0  
3        0.0  34.20      0.0      1.0      0.0      0.0      0.0  
4        0.0  26.08      0.0      1.0      0.0      0.0      0.0  
...      ...    ...      ...      ...      ...      ...      ...  
35388    NaN    NaN      NaN      NaN      NaN      NaN      NaN  
35389    NaN    NaN      NaN      NaN      NaN      NaN      NaN  
35390    NaN    NaN      NaN      NaN      NaN      NaN      NaN  
35391    NaN    NaN      NaN      NaN      NaN      NaN      NaN  
35392    NaN    NaN      NaN      NaN      NaN      NaN      NaN  

[35393 rows x 17 columns]

In [17]:
#index1=点击率/转化率
index1=[]
for i in range(size):
    if df2['转化率'][i] !=0:
        index=df2['点击率'][i]/df2['转化率'][i]
    else:
        index=0
    index1.append(index)
index_1=pd.DataFrame(index1)
df2['点击率/转化率']=index_1
df2

日期  广告形式     产品     展现量   点击量   点击率  总加购数  转化率  ROI  总订单行  \
0      2019/5/30  购物触点  SR501   893.0   4.0  0.45   0.0  0.0  0.0   0.0   
1      2019/5/30  购物触点  SR501  1327.0   8.0  0.60   1.0  0.0  0.0   0.0   
2      2019/5/29  购物触点  SR501   961.0  10.0  1.04   0.0  0.0  0.0   0.0   
3      2019/5/29  购物触点  SR501  1463.0  18.0  1.23   0.0  0.0  0.0   0.0   
4      2019/5/28  购物触点  SR501  1916.0  14.0  0.73   1.0  0.0  0.0   0.0   
...          ...   ...    ...     ...   ...   ...   ...  ...  ...   ...   
35388        NaN   NaN    NaN     NaN   NaN   NaN   NaN  NaN  NaN   NaN   
35389        NaN   NaN    NaN     NaN   NaN   NaN   NaN  NaN  NaN   NaN   
35390        NaN   NaN    NaN     NaN   NaN   NaN   NaN  NaN  NaN   NaN   
35391        NaN   NaN    NaN     NaN   NaN   NaN   NaN  NaN  NaN   NaN   
35392        NaN   NaN    NaN     NaN   NaN   NaN   NaN  NaN  NaN   NaN   

       总订单金额    总费用  是否为展位广告  是否为触点广告  是否为海投广告  是否为快车广告  是否为京东直投  点击率/转化率  
0        0.0   5.74      0.0      1.0      0.0      0.0      0.0      0.0  
1        0.0  15.51      0.0      1.0      0.0      0.0      0.0      0.0  
2        0.0  14.87      0.0      1.0      0.0      0.0      0.0      0.0  
3        0.0  34.20      0.0      1.0      0.0      0.0      0.0      0.0  
4        0.0  26.08      0.0      1.0      0.0      0.0      0.0      0.0  
...      ...    ...      ...      ...      ...      ...      ...      ...  
35388    NaN    NaN      NaN      NaN      NaN      NaN      NaN      NaN  
35389    NaN    NaN      NaN      NaN      NaN      NaN      NaN      NaN  
35390    NaN    NaN      NaN      NaN      NaN      NaN      NaN      NaN  
35391    NaN    NaN      NaN      NaN      NaN      NaN      NaN      NaN  
35392    NaN    NaN      NaN      NaN      NaN      NaN      NaN      NaN  

[35393 rows x 18 columns]

In [18]:
#index2=销售额/ROI
index2=[]
for i in range(size):
    if df2['ROI'][i] !=0:
        index=df2['总订单金额'][i]/df2['ROI'][i]
    else:
        index=0
    index2.append(index)
index_2=pd.DataFrame(index2)
df2['销售额/ROI']=index_2

In [19]:
df2

日期  广告形式     产品     展现量   点击量   点击率  总加购数  转化率  ROI  总订单行  \
0      2019/5/30  购物触点  SR501   893.0   4.0  0.45   0.0  0.0  0.0   0.0   
1      2019/5/30  购物触点  SR501  1327.0   8.0  0.60   1.0  0.0  0.0   0.0   
2      2019/5/29  购物触点  SR501   961.0  10.0  1.04   0.0  0.0  0.0   0.0   
3      2019/5/29  购物触点  SR501  1463.0  18.0  1.23   0.0  0.0  0.0   0.0   
4      2019/5/28  购物触点  SR501  1916.0  14.0  0.73   1.0  0.0  0.0   0.0   
...          ...   ...    ...     ...   ...   ...   ...  ...  ...   ...   
35388        NaN   NaN    NaN     NaN   NaN   NaN   NaN  NaN  NaN   NaN   
35389        NaN   NaN    NaN     NaN   NaN   NaN   NaN  NaN  NaN   NaN   
35390        NaN   NaN    NaN     NaN   NaN   NaN   NaN  NaN  NaN   NaN   
35391        NaN   NaN    NaN     NaN   NaN   NaN   NaN  NaN  NaN   NaN   
35392        NaN   NaN    NaN     NaN   NaN   NaN   NaN  NaN  NaN   NaN   

       总订单金额    总费用  是否为展位广告  是否为触点广告  是否为海投广告  是否为快车广告  是否为京东直投  点击率/转化率  \
0        0.0   5.74      0.0      1.0      0.0      0.0      0.0      0.0   
1        0.0  15.51      0.0      1.0      0.0      0.0      0.0      0.0   
2        0.0  14.87      0.0      1.0      0.0      0.0      0.0      0.0   
3        0.0  34.20      0.0      1.0      0.0      0.0      0.0      0.0   
4        0.0  26.08      0.0      1.0      0.0      0.0      0.0      0.0   
...      ...    ...      ...      ...      ...      ...      ...      ...   
35388    NaN    NaN      NaN      NaN      NaN      NaN      NaN      NaN   
35389    NaN    NaN      NaN      NaN      NaN      NaN      NaN      NaN   
35390    NaN    NaN      NaN      NaN      NaN      NaN      NaN      NaN   
35391    NaN    NaN      NaN      NaN      NaN      NaN      NaN      NaN   
35392    NaN    NaN      NaN      NaN      NaN      NaN      NaN      NaN   

       销售额/ROI  
0          0.0  
1          0.0  
2          0.0  
3          0.0  
4          0.0  
...        ...  
35388      NaN  
35389      NaN  
35390      NaN  
35391      NaN  
35392      NaN  

[35393 rows x 19 columns]

In [20]:
#index3=点击量/展现量
index3=[]
for i in range(size):
    if df2['展现量'][i] !=0:
        index=df2['点击量'][i]/df2['展现量'][i]
    else:
        index=0
    index3.append(index)
index_3=pd.DataFrame(index3)
df2['点击量/展现量']=index_3
df2

日期  广告形式     产品     展现量   点击量   点击率  总加购数  转化率  ROI  总订单行  \
0      2019/5/30  购物触点  SR501   893.0   4.0  0.45   0.0  0.0  0.0   0.0   
1      2019/5/30  购物触点  SR501  1327.0   8.0  0.60   1.0  0.0  0.0   0.0   
2      2019/5/29  购物触点  SR501   961.0  10.0  1.04   0.0  0.0  0.0   0.0   
3      2019/5/29  购物触点  SR501  1463.0  18.0  1.23   0.0  0.0  0.0   0.0   
4      2019/5/28  购物触点  SR501  1916.0  14.0  0.73   1.0  0.0  0.0   0.0   
...          ...   ...    ...     ...   ...   ...   ...  ...  ...   ...   
35388        NaN   NaN    NaN     NaN   NaN   NaN   NaN  NaN  NaN   NaN   
35389        NaN   NaN    NaN     NaN   NaN   NaN   NaN  NaN  NaN   NaN   
35390        NaN   NaN    NaN     NaN   NaN   NaN   NaN  NaN  NaN   NaN   
35391        NaN   NaN    NaN     NaN   NaN   NaN   NaN  NaN  NaN   NaN   
35392        NaN   NaN    NaN     NaN   NaN   NaN   NaN  NaN  NaN   NaN   

       总订单金额    总费用  是否为展位广告  是否为触点广告  是否为海投广告  是否为快车广告  是否为京东直投  点击率/转化率  \
0        0.0   5.74      0.0      1.0      0.0      0.0      0.0      0.0   
1        0.0  15.51      0.0      1.0      0.0      0.0      0.0      0.0   
2        0.0  14.87      0.0      1.0      0.0      0.0      0.0      0.0   
3        0.0  34.20      0.0      1.0      0.0      0.0      0.0      0.0   
4        0.0  26.08      0.0      1.0      0.0      0.0      0.0      0.0   
...      ...    ...      ...      ...      ...      ...      ...      ...   
35388    NaN    NaN      NaN      NaN      NaN      NaN      NaN      NaN   
35389    NaN    NaN      NaN      NaN      NaN      NaN      NaN      NaN   
35390    NaN    NaN      NaN      NaN      NaN      NaN      NaN      NaN   
35391    NaN    NaN      NaN      NaN      NaN      NaN      NaN      NaN   
35392    NaN    NaN      NaN      NaN      NaN      NaN      NaN      NaN   

       销售额/ROI   点击量/展现量  
0          0.0  0.004479  
1          0.0  0.006029  
2          0.0  0.010406  
3          0.0  0.012303  
4          0.0  0.007307  
...        ...       ...  
35388      NaN       NaN  
35389      NaN       NaN  
35390      NaN       NaN  
35391      NaN       NaN  
35392      NaN       NaN  

[35393 rows x 20 columns]

In [21]:
#index4=加购数/点击量
index4=[]
for i in range(size):
    if df2['点击量'][i] !=0:
        index=df2['总加购数'][i]/df2['点击量'][i]
    else:
        index=0
    index4.append(index)
index_4=pd.DataFrame(index4)
df2['加购数/点击量']=index_4
df2

日期  广告形式     产品     展现量   点击量   点击率  总加购数  转化率  ROI  总订单行  ...  \
0      2019/5/30  购物触点  SR501   893.0   4.0  0.45   0.0  0.0  0.0   0.0  ...   
1      2019/5/30  购物触点  SR501  1327.0   8.0  0.60   1.0  0.0  0.0   0.0  ...   
2      2019/5/29  购物触点  SR501   961.0  10.0  1.04   0.0  0.0  0.0   0.0  ...   
3      2019/5/29  购物触点  SR501  1463.0  18.0  1.23   0.0  0.0  0.0   0.0  ...   
4      2019/5/28  购物触点  SR501  1916.0  14.0  0.73   1.0  0.0  0.0   0.0  ...   
...          ...   ...    ...     ...   ...   ...   ...  ...  ...   ...  ...   
35388        NaN   NaN    NaN     NaN   NaN   NaN   NaN  NaN  NaN   NaN  ...   
35389        NaN   NaN    NaN     NaN   NaN   NaN   NaN  NaN  NaN   NaN  ...   
35390        NaN   NaN    NaN     NaN   NaN   NaN   NaN  NaN  NaN   NaN  ...   
35391        NaN   NaN    NaN     NaN   NaN   NaN   NaN  NaN  NaN   NaN  ...   
35392        NaN   NaN    NaN     NaN   NaN   NaN   NaN  NaN  NaN   NaN  ...   

         总费用  是否为展位广告  是否为触点广告  是否为海投广告  是否为快车广告  是否为京东直投  点击率/转化率  销售额/ROI  \
0       5.74      0.0      1.0      0.0      0.0      0.0      0.0      0.0   
1      15.51      0.0      1.0      0.0      0.0      0.0      0.0      0.0   
2      14.87      0.0      1.0      0.0      0.0      0.0      0.0      0.0   
3      34.20      0.0      1.0      0.0      0.0      0.0      0.0      0.0   
4      26.08      0.0      1.0      0.0      0.0      0.0      0.0      0.0   
...      ...      ...      ...      ...      ...      ...      ...      ...   
35388    NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
35389    NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
35390    NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
35391    NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
35392    NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN   

        点击量/展现量   加购数/点击量  
0      0.004479  0.000000  
1      0.006029  0.125000  
2      0.010406  0.000000  
3      0.012303  0.000000  
4      0.007307  0.071429  
...         ...       ...  
35388       NaN       NaN  
35389       NaN       NaN  
35390       NaN       NaN  
35391       NaN       NaN  
35392       NaN       NaN  

[35393 rows x 21 columns]

In [22]:
#index5=订单数/加购数
index5=[]
for i in range(size):
    if df2['总订单行'][i] !=0:
        index=df2['总加购数'][i]/df2['总订单行'][i]
    else:
        index=0
    index5.append(index)
index_5=pd.DataFrame(index5)
df2['订单数/加购数']=index_5
df2

日期  广告形式     产品     展现量   点击量   点击率  总加购数  转化率  ROI  总订单行  ...  \
0      2019/5/30  购物触点  SR501   893.0   4.0  0.45   0.0  0.0  0.0   0.0  ...   
1      2019/5/30  购物触点  SR501  1327.0   8.0  0.60   1.0  0.0  0.0   0.0  ...   
2      2019/5/29  购物触点  SR501   961.0  10.0  1.04   0.0  0.0  0.0   0.0  ...   
3      2019/5/29  购物触点  SR501  1463.0  18.0  1.23   0.0  0.0  0.0   0.0  ...   
4      2019/5/28  购物触点  SR501  1916.0  14.0  0.73   1.0  0.0  0.0   0.0  ...   
...          ...   ...    ...     ...   ...   ...   ...  ...  ...   ...  ...   
35388        NaN   NaN    NaN     NaN   NaN   NaN   NaN  NaN  NaN   NaN  ...   
35389        NaN   NaN    NaN     NaN   NaN   NaN   NaN  NaN  NaN   NaN  ...   
35390        NaN   NaN    NaN     NaN   NaN   NaN   NaN  NaN  NaN   NaN  ...   
35391        NaN   NaN    NaN     NaN   NaN   NaN   NaN  NaN  NaN   NaN  ...   
35392        NaN   NaN    NaN     NaN   NaN   NaN   NaN  NaN  NaN   NaN  ...   

       是否为展位广告  是否为触点广告  是否为海投广告  是否为快车广告  是否为京东直投  点击率/转化率  销售额/ROI  \
0          0.0      1.0      0.0      0.0      0.0      0.0      0.0   
1          0.0      1.0      0.0      0.0      0.0      0.0      0.0   
2          0.0      1.0      0.0      0.0      0.0      0.0      0.0   
3          0.0      1.0      0.0      0.0      0.0      0.0      0.0   
4          0.0      1.0      0.0      0.0      0.0      0.0      0.0   
...        ...      ...      ...      ...      ...      ...      ...   
35388      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
35389      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
35390      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
35391      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
35392      NaN      NaN      NaN      NaN      NaN      NaN      NaN   

        点击量/展现量   加购数/点击量  订单数/加购数  
0      0.004479  0.000000      0.0  
1      0.006029  0.125000      0.0  
2      0.010406  0.000000      0.0  
3      0.012303  0.000000      0.0  
4      0.007307  0.071429      0.0  
...         ...       ...      ...  
35388       NaN       NaN      NaN  
35389       NaN       NaN      NaN  
35390       NaN       NaN      NaN  
35391       NaN       NaN      NaN  
35392       NaN       NaN      NaN  

[35393 rows x 22 columns]

In [ ]:
#描述性统计

In [31]:
dependent = df2.loc[:,['点击率/转化率','销售额/ROI','点击量/展现量','加购数/点击量','订单数/加购数']]
independent = df2.loc[:,['是否为展位广告','是否为触点广告','是否为海投广告','是否为快车广告','是否为京东直投','总费用']]

In [32]:
dependent.describe()

点击率/转化率       销售额/ROI       点击量/展现量       加购数/点击量       订单数/加购数
count  33049.000000  32268.000000  33049.000000  33049.000000  33049.000000
mean       0.144434     69.180545      0.024409      0.104120      0.633639
std        0.957063    451.518793      0.083388      0.625749      2.840181
min        0.000000      0.000000      0.000000      0.000000      0.000000
25%        0.000000      0.000000      0.000000      0.000000      0.000000
50%        0.000000      0.000000      0.000000      0.000000      0.000000
75%        0.000000      0.000000      0.013550      0.000000      0.000000
max       59.775000  19426.821705      4.000000     59.000000    110.000000

In [33]:
independent.describe()

是否为展位广告       是否为触点广告       是否为海投广告       是否为快车广告       是否为京东直投  \
count  33049.000000  33049.000000  33049.000000  33049.000000  33049.000000   
mean       0.045357      0.036915      0.034434      0.857030      0.026264   
std        0.208089      0.188556      0.182343      0.350047      0.159922   
min        0.000000      0.000000      0.000000      0.000000      0.000000   
25%        0.000000      0.000000      0.000000      1.000000      0.000000   
50%        0.000000      0.000000      0.000000      1.000000      0.000000   
75%        0.000000      0.000000      0.000000      1.000000      0.000000   
max        1.000000      1.000000      1.000000      1.000000      1.000000   

                总费用  
count  32181.000000  
mean     110.991053  
std      480.903972  
min        0.000000  
25%        0.000000  
50%        0.000000  
75%       35.060000  
max    19428.680000

In [30]:
#生成输出文档
import csv
outfile=open('after.csv','w',newline='')
to_write=csv.writer(outfile)
title=['日期','广告形式','产品','展现量','点击量', '点击率','总加购数','转化率','ROI','总订单行','总订单金额','总费用',
       '是否为展位广告','是否为触点广告','是否为海投广告','是否为快车广告','是否为京东直投', 
       '点击率/转化率','销售额/ROI','点击量/展现量','加购数/点击量','订单数/加购数']
to_write.writerow(title)
for i in range(size):
    row = df2.iloc[i].values.tolist()
    to_write.writerow(row)
outfile.close()

**以下为PCA代码，不需要，但是保留着，万一有用呢**

In [14]:
#df3=df2.iloc[:,13:]
#df3

点击量/展现量   加购数/点击量  订单数/加购数
0      0.004479  0.000000      0.0
1      0.006029  0.125000      0.0
2      0.010406  0.000000      0.0
3      0.012303  0.000000      0.0
4      0.007307  0.071429      0.0
...         ...       ...      ...
33044  0.333333  0.000000      0.0
33045  0.008434  0.016260     12.0
33046  0.000000  0.000000      0.0
33047  0.027506  0.001350      0.0
33048  0.000000  0.000000      0.0

[33049 rows x 3 columns]

In [15]:
#PCA
#df3_T = df3.T
#df3_T

0         1         2         3         4         5         6      \
点击量/展现量  0.004479  0.006029  0.010406  0.012303  0.007307  0.004558  0.008522   
加购数/点击量  0.000000  0.125000  0.000000  0.000000  0.071429  0.800000  0.050000   
订单数/加购数  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  1.000000   

            7        8      9      ...     33039     33040     33041  33042  \
点击量/展现量  0.011516  0.00202    0.0  ...  0.032895  0.011228  0.009384    0.0   
加购数/点击量  0.083333  4.00000    0.0  ...  0.006192  0.000000  0.002225    0.0   
订单数/加购数  0.000000  0.00000    0.0  ...  0.000000  0.000000  0.000000    0.0   

           33043     33044      33045  33046     33047  33048  
点击量/展现量  0.02337  0.333333   0.008434    0.0  0.027506    0.0  
加购数/点击量  0.00000  0.000000   0.016260    0.0  0.001350    0.0  
订单数/加购数  0.00000  0.000000  12.000000    0.0  0.000000    0.0  

[3 rows x 33049 columns]

In [16]:
#from sklearn import decomposition
#pca = decomposition.PCA(n_components=1)
#pca.fit(df3_T)
#print(pca.transform(df3_T))
#print(pca.explained_variance_ratio_)

[[-176.39928398]
 [-170.76988765]
 [ 347.16917162]]
[0.96580412]


In [17]:
#index6=PCA结果
#index6=[]
#for i in range(size):
#    pca=df3['点击量/展现量'][i]*(-176.4)+df3['加购数/点击量'][i]*(-170.8)+df3['订单数/加购数'][i]*(347.2)
#    index6.append(pca)
#index_6=pd.DataFrame(index6)
#df2['pca']=index_6
#df2

日期  广告形式     产品      展现量    点击量   点击率  总加购数  转化率   ROI  总订单行  \
0      2019/5/30  购物触点  SR501    893.0    4.0  0.45   0.0  0.0  0.00   0.0   
1      2019/5/30  购物触点  SR501   1327.0    8.0  0.60   1.0  0.0  0.00   0.0   
2      2019/5/29  购物触点  SR501    961.0   10.0  1.04   0.0  0.0  0.00   0.0   
3      2019/5/29  购物触点  SR501   1463.0   18.0  1.23   0.0  0.0  0.00   0.0   
4      2019/5/28  购物触点  SR501   1916.0   14.0  0.73   1.0  0.0  0.00   0.0   
...          ...   ...    ...      ...    ...   ...   ...  ...   ...   ...   
33044  2019/12/2  京东直投   京东直投      6.0    2.0  0.33   0.0  0.0  0.00   0.0   
33045  2019/12/1  京东直投   京东直投  87498.0  738.0  0.01  12.0  0.0  0.28   1.0   
33046  2019/12/2  京东直投   京东直投     34.0    0.0  0.00   0.0  0.0  0.00   0.0   
33047  2019/12/1  京东直投   京东直投  26940.0  741.0  0.03   1.0  0.0  0.00   0.0   
33048  2019/12/2  京东直投   京东直投      8.0    0.0  0.00   0.0  0.0  0.00   0.0   

       总订单金额  点击率/转化率     销售额/ROI   点击量/展现量   加购数/点击量  订单数/加购数          pca  
0        0.0      0.0    0.000000  0.004479  0.000000      0.0    -0.790146  
1        0.0      0.0    0.000000  0.006029  0.125000      0.0   -22.413451  
2        0.0      0.0    0.000000  0.010406  0.000000      0.0    -1.835588  
3        0.0      0.0    0.000000  0.012303  0.000000      0.0    -2.170335  
4        0.0      0.0    0.000000  0.007307  0.071429      0.0   -13.488935  
...      ...      ...         ...       ...       ...      ...          ...  
33044    0.0      0.0    0.000000  0.333333  0.000000      0.0   -58.800000  
33045   79.0      0.0  282.142857  0.008434  0.016260     12.0  4162.134922  
33046    0.0      0.0    0.000000  0.000000  0.000000      0.0     0.000000  
33047    0.0      0.0    0.000000  0.027506  0.001350      0.0    -5.082482  
33048    0.0      0.0    0.000000  0.000000  0.000000      0.0     0.000000  

[33049 rows x 17 columns]

In [18]:
#df3.describe()

点击量/展现量       加购数/点击量       订单数/加购数
count  33049.000000  33049.000000  33049.000000
mean       0.024409      0.104120      0.633639
std        0.083388      0.625749      2.840181
min        0.000000      0.000000      0.000000
25%        0.000000      0.000000      0.000000
50%        0.000000      0.000000      0.000000
75%        0.013550      0.000000      0.000000
max        4.000000     59.000000    110.000000

In [ ]:
#选取 点击量/展现量